### Импорт библов

In [1]:
import spacy
import pandas as pd
import nltk
from nltk import word_tokenize
import re
import string
import numpy as np
import tqdm
from pymystem3 import Mystem
mystem = Mystem() 
import re
import nltk
import numpy as np
import pandas as pd
from gensim.models import Word2Vec
from collections import Counter 
from nltk import word_tokenize
from nltk.corpus import stopwords
from sklearn.cluster import MiniBatchKMeans
from sklearn.metrics import silhouette_samples, silhouette_score
from sklearn.utils import shuffle
import string
lst_stopwords = nltk.corpus.stopwords.words('russian')
lst_stopwords.extend(['…', '«', '»', '...'])
nlp = spacy.load('ru_core_news_sm')

### Прочти данные в таком виде

In [2]:
df=pd.read_excel('data.csv')
df

,text
0,Фото: Виктор Чумаков / Пресс-служба судостроит...
1,Фото: Drew Coffman / Unsplash Андрей Ставицки...
2,Фото: TECHSPOT Андрей Ставицкий Китайская комп...
3,ото: Pixabay Марина Совина Бывший сотрудник Бе...
4,Фото: Kay Nietfeld / Globallookpress.com Марин...


### функции предобраб текста

In [3]:
def clean_text(text, tokenizer, stopwords):

    text = str(text).lower()  
    text = re.sub(r"\[(.*?)\]", "", text)  
    text = re.sub(r"\s+", " ", text)  
    text = re.sub(r"\w+…|…", "", text)  
    text = re.sub(r"(?<=\w)-(?=\w)", " ", text)  
    text = re.sub(
        f"[{re.escape(string.punctuation)}]", "", text
    )  

    tokens = tokenizer(text)  
    tokens = [t for t in tokens if not t in lst_stopwords]  
    tokens = ["" if t.isdigit() else t for t in tokens]  
    tokens = [t for t in tokens if len(t) > 1] 
    return tokens
def prep_tokens(df_raw):
    text_columns = ['text']
    #df_raw['content'] = df_raw['content'].fillna(" ")
    for col in text_columns:
        df_raw[col] = df_raw[col].astype(str)
    # создаем текст основанный на content title и tag
    df_raw["text"] = df_raw[text_columns].apply(lambda x: " | ".join(x), axis=1)
    df_raw["tokens"] = df_raw["text"].map(lambda x: clean_text(x, word_tokenize, lst_stopwords))
    _, idx = np.unique(df_raw["tokens"], return_index=True)
    df_raw = df_raw.iloc[idx, :]

    # Remove empty values
    df_raw = df_raw.loc[df_raw.tokens.map(lambda x: len(x) > 0), ["text", "tokens"]]
    return df_raw

### Получи токены

In [4]:
prep_data=prep_tokens(df)
prep_data

,text,tokens
3,ото: Pixabay Марина Совина Бывший сотрудник Бе...,"[ото, pixabay, марина, совина, бывший, сотрудн..."
1,Фото: Drew Coffman / Unsplash Андрей Ставицки...,"[фото, drew, coffman, unsplash, андрей, ставиц..."
4,Фото: Kay Nietfeld / Globallookpress.com Марин...,"[фото, kay, nietfeld, globallookpresscom, мари..."
2,Фото: TECHSPOT Андрей Ставицкий Китайская комп...,"[фото, techspot, андрей, ставицкий, китайская,..."
0,Фото: Виктор Чумаков / Пресс-служба судостроит...,"[фото, виктор, чумаков, пресс, служба, судостр..."


In [5]:
###создаем словарь

In [6]:

docs = prep_data["text"].values
tokenized_docs = prep_data["tokens"].values
vocab = Counter()
for token in tokenized_docs:
    vocab.update(token)


### Векторизируем слова

In [7]:
def vectorize(list_of_docs, model):
    features = []

    for tokens in list_of_docs:
        zero_vector = np.zeros(model.vector_size)
        vectors = []
        for token in tokens:
            if token in model.wv:
                try:
                    vectors.append(model.wv[token])
                except KeyError:
                    continue
        if vectors:
            vectors = np.asarray(vectors)
            avg_vec = vectors.mean(axis=0)
            features.append(avg_vec)
        else:
            features.append(zero_vector)
    return features

In [8]:
model = Word2Vec(sentences=prep_data['tokens'].values, vector_size=100, window=5, min_count=1, workers=4)

In [9]:
vectorized_docs= vectorize(prep_data['tokens'], model=model)
len(vectorized_docs), len(vectorized_docs[0])

(5, 100)

In [10]:
prep_data['vectors'] =vectorized_docs
prep_data.fillna('')

,text,tokens,vectors
3,ото: Pixabay Марина Совина Бывший сотрудник Бе...,"[ото, pixabay, марина, совина, бывший, сотрудн...","[-0.0009992489, 0.0013703133, -0.0002628915, 0..."
1,Фото: Drew Coffman / Unsplash Андрей Ставицки...,"[фото, drew, coffman, unsplash, андрей, ставиц...","[0.00012650588, -0.0009367504, -0.00037640653,..."
4,Фото: Kay Nietfeld / Globallookpress.com Марин...,"[фото, kay, nietfeld, globallookpresscom, мари...","[7.635207e-05, -0.00078705803, 0.0006089282, -..."
2,Фото: TECHSPOT Андрей Ставицкий Китайская комп...,"[фото, techspot, андрей, ставицкий, китайская,...","[-0.00031375713, 0.00039977144, 0.00052372174,..."
0,Фото: Виктор Чумаков / Пресс-служба судостроит...,"[фото, виктор, чумаков, пресс, служба, судостр...","[-9.7775526e-05, 0.0007715449, -0.0001367061, ..."


### получаем класетры 

In [11]:
def mbkmeans_clusters(X, k, mb=500, print_silhouette_values=False):

    km = MiniBatchKMeans(n_clusters=k, batch_size=mb).fit(X)
    print(f"For n_clusters = {k}")
    print(f"Silhouette coefficient: {silhouette_score(X, km.labels_):0.2f}")
    print(f"Inertia:{km.inertia_}")

    if print_silhouette_values:
        sample_silhouette_values = silhouette_samples(X, km.labels_)
        print(f"Silhouette values:")
        silhouette_values = []
        for i in range(k):
            cluster_silhouette_values = sample_silhouette_values[km.labels_ == i]
            silhouette_values.append(
                (
                    i,
                    cluster_silhouette_values.shape[0],
                    cluster_silhouette_values.mean(),
                    cluster_silhouette_values.min(),
                    cluster_silhouette_values.max(),
                )
            )
        silhouette_values = sorted(
            silhouette_values, key=lambda tup: tup[2], reverse=True
        )
        for s in silhouette_values:
            print(
                f"    Cluster {s[0]}: Size:{s[1]} | Avg:{s[2]:.2f} | Min:{s[3]:.2f} | Max: {s[4]:.2f}"
            )
    return km, km.labels_

In [12]:
### функция получения кластеров k - количествоо кластеров, X данные

In [13]:

clustering, cluster_labels = mbkmeans_clusters(prep_data['vectors'].tolist(),k=3, print_silhouette_values=True)
df_clusters = pd.DataFrame({
    "text": prep_data["text"].values,
    "tokens": [" ".join(text) for text in tokenized_docs],
    "cluster": cluster_labels
})


For n_clusters = 3
Silhouette coefficient: -0.04
Inertia:8.285716700170844e-05
Silhouette values:
    Cluster 1: Size:1 | Avg:0.00 | Min:0.00 | Max: 0.00
    Cluster 2: Size:1 | Avg:0.00 | Min:0.00 | Max: 0.00
    Cluster 0: Size:3 | Avg:-0.07 | Min:-0.08 | Max: -0.05


C:\Users\Jora\Desktop\hack_0912\venv\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 3 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [14]:
print("Самые популярные темы в кластере:")
k=3 # количесвто класетров ходных
slov1={}
for i in range(k):
    tokens_per_cluster = ""
    most_representative = model.wv.most_similar(positive=[clustering.cluster_centers_[i]], topn=5)
    for t in most_representative:
        tokens_per_cluster += f"{t[0]} "
        slov1[i]=tokens_per_cluster
print(slov1)


Самые популярные темы в кластере:
{0: 'министерстве герц преступлениями украине это ', 1: 'apple смартфонов ставицкий базе nothing ', 2: 'заливе адмирал игорь фрегат обороны '}


In [15]:
docs = prep_data["text"].values
tokenized_docs = prep_data["tokens"].values
vocab = Counter()
for token in tokenized_docs:
    vocab.update(token)

test_cluster = 1
most_representative_docs = np.argsort(
    np.linalg.norm(vectorized_docs - clustering.cluster_centers_[test_cluster], axis=1)
)
for d in most_representative_docs[:10]:
    doc = nlp(docs[d])

# в переменной 'doc' теперь содержится обработанная версия текста
# мы можем делать с ней все что угодно!
# например, распечатать все обнаруженные именованные сущности
    for entity in doc.ents:
        print(f"{entity.text} ({entity.label_})")
    print("-------------")

Андрей Ставицкий (PER)
Основатель (PER)
Nothing (ORG)
Apple (ORG)
CNBC (ORG)
Nothing (ORG)
OnePlus (ORG)
Карл Пэй (PER)
Apple (ORG)
IT-гиганта (ORG)
Apple (ORG)
-------------
Виктор Чумаков (PER)
Северная верфь (ORG)
РИА Новости (ORG)
Даниил Иринин Фрегат (PER)
Финском заливе (LOC)
Балтийского моря (LOC)
Министерстве обороны (ORG)
России (LOC)
РИА Новости (ORG)
Военно-морским флотом (ORG)
Николаю Евменову (PER)
Северная верфь (ORG)
Балтийского флота (ORG)
ТАСС (ORG)
Северного флота (ORG)
России (LOC)
Адмирал Горшков (PER)
Кронштадтский морской завод (ORG)
Северной верфи (ORG)
Игорь Орлов (PER)
-------------
TECHSPOT (ORG)
Андрей Ставицкий (PER)
BOE (ORG)
TechSpot (ORG)
Китае (LOC)
BOE (ORG)
BOE (ORG)
AMD Ryzen (ORG)
Nvidia RTX (ORG)
Valorant (ORG)
TechSpot (ORG)
BOE (ORG)
Asus (ORG)
Nvidia (ORG)
Чек Индекс (ORG)
ЮMoney (ORG)
России (LOC)
-------------
Марина Совина (PER)
Луганской народной республики (ЛНР) (LOC)
Совместном центре контроля и координации вопросов (ORG)
Украины (СЦКК) (LO

In [16]:
!pip freeze > requirements.txt